In [35]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import col

In [6]:
spark = SparkContext(master="local", appName="proyecto-spark")
sql_context = SQLContext(spark)

21/08/09 12:03:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [12]:
deporte = sql_context.read.csv("files/deporte.csv", inferSchema=True, header=True)
deporte.show(4)

+----------+----------+
|deporte_id|   deporte|
+----------+----------+
|         1|Basketball|
|         2|      Judo|
|         3|  Football|
|         4|Tug-Of-War|
+----------+----------+
only showing top 4 rows



In [20]:
deportista = sql_context.read.csv("files/deportista.csv", inferSchema=True, header=True).union(
  sql_context.read.csv("files/deportista.csv")
)
deportista.show(4)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
+-------------+--------------------+------+----+------+----+---------+
only showing top 4 rows



In [22]:
deportista_error = sql_context.read.csv("files/deportistaError.csv", inferSchema=True, header=True)
deportista_error.show(4)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|  null|null|      273|
|            4|Edgar Lindenau Aabye|     1|  34|  null|null|      278|
+-------------+--------------------+------+----+------+----+---------+
only showing top 4 rows



In [23]:
evento = sql_context.read.csv("files/evento.csv", inferSchema=True, header=True)
evento.show(4)

+---------+--------------------+----------+
|evento_id|              evento|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
|        4|Tug-Of-War Men's ...|         4|
+---------+--------------------+----------+
only showing top 4 rows



In [27]:
juegos = sql_context.read.csv("files/juegos.csv", header=True).withColumnRenamed("_c0", "juego_id")
juegos.show(4)

+--------+------------+-----+---------+---------+
|juego_id|nombre_juego|annio|temporada|   ciudad|
+--------+------------+-----+---------+---------+
|       1| 1896 Verano| 1896|   Verano|   Athina|
|       2| 1900 Verano| 1900|   Verano|    Paris|
|       3| 1904 Verano| 1904|   Verano|St. Louis|
|       4| 1906 Verano| 1906|   Verano|   Athina|
+--------+------------+-----+---------+---------+
only showing top 4 rows



21/08/09 12:19:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , nombre_juego, annio, temporada, ciudad
 Schema: _c0, nombre_juego, annio, temporada, ciudad
Expected: _c0 but found: 
CSV file: file:///home/will/Projects/spark/files/juegos.csv


In [29]:
paises = sql_context.read.csv("files/paises.csv", inferSchema=True, header=True)
paises.show(4)

+---+--------------------+-----+
| id|              equipo|sigla|
+---+--------------------+-----+
|  1|         30. Februar|  AUT|
|  2|A North American ...|  MEX|
|  3|           Acipactli|  MEX|
|  4|             Acturus|  ARG|
+---+--------------------+-----+
only showing top 4 rows



In [30]:
resultados = sql_context.read.csv("files/resultados.csv", inferSchema=True, header=True)
resultados.show(4)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
+------------+-------+-------------+--------+---------+
only showing top 4 rows



In [34]:
deportista = deportista.withColumnRenamed("genero", "sexo").drop("altura")
deportista.printSchema()

root
 |-- deportista_id: string (nullable = true)
 |-- nombre: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: string (nullable = true)
 |-- peso: string (nullable = true)
 |-- equipo_id: string (nullable = true)



In [39]:
deportista.select("deportista_id", "nombre", col("edad").alias("edadAlJugar"), "equipo_id") \
  .filter(
    (deportista.edad != 0)
  ) \
  .sort("edadAlJugar") \
  .show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|        22411|Magdalena Cecilia...|         11|      413|
|        40129|    Luigina Giavotti|         11|      507|
|        37333|Carlos Bienvenido...|         11|      982|
|        47618|Sonja Henie Toppi...|         11|      742|
|        51268|      Beatrice Hutiu|         11|      861|
+-------------+--------------------+-----------+---------+
only showing top 5 rows



In [5]:
spark.stop()